# From Prev.
#Rearranged code
#Used larger training set
#Match Similar words manually (due to limited size of categories [hopefully :P])

In [50]:
import pandas as pd
from urllib.parse import unquote

df = pd.read_json('data-1.json',encoding='utf-8')
#df.drop(['date','body','author','url'],axis=1,inplace=True) #for test_2.json
#df.drop(['id','news_content','date','url'],axis=1,inplace=True)

In [51]:
CAT= {
    #category: change to this
    'ข่าวบันเทิง':"entertainment",
    'regional':'local',
    'sport':'sports',
    'ข่าวกีฬา':'sports',
    'ข่าวอาชญากรรม':'crime',
    'khrongkhaidaasapparod':'ข่าวโครงข่ายตาสับปะรด',
    'around-the-world-news':'foreign',
    'abroad':'foreign',
    'ข่าวต่างประเทศ':'foreign',
    'ข่าวไลฟ์สไตล์':'lifestyle',
    'analysis-today-politics':'politics',
    'politic':'politics',
    'ข่าวเศรษฐกิจ':'economics',
    'economy':'economics',
    'economic':'economics',
    'uncategorized':'others',
    'auto':'car-vehicle',
    'thai-soccer':'thaifootball',
    'ข่าวพระราชสำนัก':'royalnews',
    'it':'tech'
}

In [52]:
def conv_cate(category):
    result = CAT.get(category,-1)
    if result!=-1:
        return result
    else :
        #in case want to update dictionary
        #CAT[category] = category
        return category

In [53]:
df['category'] = df['category'].apply(unquote).apply(conv_cate)

In [54]:
#tags cleaning
#df[df['category']=='1293427']
#df.drop([292],inplace=True)
df['category'].unique()
len(df['category'].unique())

53

In [55]:
import numpy as np
category = np.array(df['category'])
tags, freq = np.unique(category,return_counts=True)
freqList = list(zip(tags,freq))
freqList.sort(key=lambda elem:elem[1]) #sort by frequency
freqList[20:]

[('car-vehicle', 5),
 ('stock-monitor', 5),
 ('thaifootball', 6),
 ('covid-19', 7),
 ('index.php', 7),
 ('ข่าวโครงข่ายตาสับปะรด', 9),
 ('life', 10),
 ('women', 14),
 ('ข่าวในประเทศ', 14),
 ('bangkok', 15),
 ('bbc-thai', 18),
 ('trcheerthai', 18),
 ('special-stories', 20),
 ('society', 21),
 ('lifestyle', 22),
 ('tech', 22),
 ('breaking-news', 23),
 ('ข่าวด่วน', 23),
 ('pr-news', 28),
 ('news', 29),
 ('education', 30),
 ('others', 31),
 ('around-thailand', 32),
 ('business', 34),
 ('eurofootball', 78),
 ('crime', 80),
 ('economics', 96),
 ('article', 98),
 ('entertainment', 129),
 ('sports', 257),
 ('politics', 261),
 ('foreign', 338),
 ('local', 357)]

In [56]:
df.head(1)

,id,news_topic,news_content,date,author,url,category
0,1,"""นกเอี้ยง จันทิมา""โดนหนุ่มจับก้น เจ้าตัวตบสวนถ...",นกเอี้ยง จันทิมา นักร้องสาว โพสต์เฟสบุ๊กส่วนตั...,2019-06-29 18:30:44,ch3,http://news.ch3thailand.com/entertainment/98347,entertainment


In [57]:

X = np.array(df['news_topic'])
y = np.array(df['category'])

In [58]:
#y cleaning


In [59]:
import re
#prepare each sentence
def text_prepare(text):
    text = re.sub("[/(){}\[\]\|@,;\n\t:\'\- ]", '', text)
    text = re.sub("[/\xa0]", '', text)
    return text

In [60]:
#execute prepare
X = np.array([text_prepare(title) for title in X])

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [62]:
type(X_train)

numpy.ndarray

In [63]:
len(X_test)

545

In [64]:
#not well distributed
tags, freq = np.unique(y_train,return_counts=True)
list(zip(tags,freq))

[('advertorial', 4),
 ('agriculture', 5),
 ('around-thailand', 21),
 ('article', 79),
 ('bangkok', 12),
 ('bbc-thai', 15),
 ('beauty', 2),
 ('boxing-mma', 1),
 ('breaking-news', 12),
 ('business', 25),
 ('car-vehicle', 3),
 ('covid-19', 6),
 ('crime', 65),
 ('drama', 1),
 ('economics', 73),
 ('education', 22),
 ('entertainment', 94),
 ('eurofootball', 59),
 ('food', 2),
 ('foreign', 250),
 ('hot-topics', 1),
 ('house-condo', 2),
 ('in-the-news', 1),
 ('index.php', 6),
 ('life', 9),
 ('lifestyle', 15),
 ('live-from-the-scene', 1),
 ('local', 269),
 ('monitor-news', 2),
 ('news', 25),
 ('online-social', 2),
 ('others', 22),
 ('politics', 193),
 ('pr-news', 21),
 ('society', 14),
 ('special-stories', 15),
 ('sports', 196),
 ('stock-monitor', 4),
 ('tech', 18),
 ('thaifootball', 4),
 ('travel', 1),
 ('trcheerthai', 11),
 ('volleyball', 2),
 ('women', 11),
 ('worldcup', 2),
 ('ข่าวด่วน', 21),
 ('ข่าวโครงข่ายตาสับปะรด', 4),
 ('ข่าวในประเทศ', 9),
 ('ดวง', 1),
 ('หวย', 2)]

### Pythainlp sentence vectorizer

In [65]:
from pythainlp.word_vector import sentence_vectorizer
#sentence_vectorizer(sentence, use_mean=True)
#returns 2d array

In [66]:
X_train_sv= np.array([sentence_vectorizer(text,use_mean=True)[0] for text in X_train])

In [67]:
X_train_sv

array([[ 0.39712699, -0.07797   , -0.19719366, ...,  0.10240766,
        -0.067172  , -0.13942533],
       [-0.33991815, -0.11597231,  0.04596185, ..., -0.01522454,
        -0.09269116,  0.16731854],
       [-0.20230964,  0.135857  ,  0.06884378, ...,  0.18333993,
        -0.05423329,  0.16160221],
       ...,
       [-0.1430811 , -0.09691   ,  0.1751326 , ...,  0.1083183 ,
         0.0437399 ,  0.0330731 ],
       [-0.0891384 ,  0.01231853,  0.1303668 , ...,  0.07349853,
        -0.00351393,  0.17122454],
       [-0.0050979 ,  0.1312047 ,  0.0795295 , ...,  0.1032143 ,
         0.1377731 ,  0.1371747 ]])

In [68]:
#turn y values to 1s,0s

In [69]:
# from sklearn.preprocessing import MultiLabelBinarizer
# this is for list of tags

In [70]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
#lb.fit_transform([1, 2, 6, 4, 2]) 
#fit & transform in one step
#fit = figure out different classes and turn into necessary binary formats
#transform = transform given data into binary formats

In [71]:
y_train_lb = lb.fit_transform(y_train)


In [72]:
lb.classes_.size

50

In [73]:
#for illustration & checking
Y_LB = pd.DataFrame(y_train_lb,columns = lb.classes_)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
Y_LB.iloc[10:20]
#y_train.iloc[10:20]

,advertorial,agriculture,around-thailand,article,bangkok,bbc-thai,beauty,boxing-mma,breaking-news,business,car-vehicle,covid-19,crime,drama,economics,education,entertainment,eurofootball,food,foreign,hot-topics,house-condo,in-the-news,index.php,life,lifestyle,live-from-the-scene,local,monitor-news,news,online-social,others,politics,pr-news,society,special-stories,sports,stock-monitor,tech,thaifootball,travel,trcheerthai,volleyball,women,worldcup,ข่าวด่วน,ข่าวโครงข่ายตาสับปะรด,ข่าวในประเทศ,ดวง,หวย
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
y_train_lb.shape

(1635, 50)

In [75]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#example
X = np.array([
     [10, 10],
     [8, 10],
     [-5, 5.5],
     [-5.4, 5.5],
     [-20, -20],
     [-15, -20]])
y = np.array([0, 0, 1, 1, 2, 2])
clf = OneVsRestClassifier(SVC()).fit(X, y)
clf.predict([[-19, -20], [9, 9], [-5, 5]])

array([2, 0, 1])

In [76]:
clf = OneVsRestClassifier(SVC()).fit(X_train_sv, y_train)

In [77]:
y_train

array(['lifestyle', 'entertainment', 'local', ..., 'economics', 'crime',
       'bangkok'], dtype=object)

In [78]:
#check the predictions, models applied correctly? Yes
y = clf.predict(X_train_sv)
print(np.concatenate((y.reshape(len(y),1), y_train.reshape(len(y_train),1)),1))

[['lifestyle' 'lifestyle']
 ['entertainment' 'entertainment']
 ['local' 'local']
 ...
 ['economics' 'economics']
 ['crime' 'crime']
 ['bangkok' 'bangkok']]


In [79]:
X_test = np.array([text_prepare(title) for title in X_test])
X_test_sv = np.array([sentence_vectorizer(text,use_mean=True)[0] for text in X_test])
y_pred = clf.predict(X_test_sv)
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#print(np.concatenate(y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1),1))

[['economics' 'economics']
 ['local' 'crime']
 ['foreign' 'foreign']
 ...
 ['local' 'local']
 ['politics' 'politics']
 ['foreign' 'foreign']]


In [80]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 6 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


0.48990825688073397

In [81]:
test = text_prepare('“อนุทิน” ขอ ผู้ร้อง เปิดชื่อ อักษรย่อ \"พ\" คนหักหัวคิวโรงแรม 40%')
test = np.array([sentence_vectorizer(test, use_mean=True)[0]])
clf.predict(test)

array(['politics'], dtype='<U21')